<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-LCC/blob/main/Tareas/TC-09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Trabajo en clase 9</h1>

<h2>Topic Modelling</h1>

Haremos Topic Modelling usando varias técnicas:

* Clustering en representaciones vectoriales de documentos BOW y TF-IDF.
* LSA, implementación en [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

Posteriormente, usaras:
* LDA. Implementación en [gensim](https://radimrehurek.com/gensim/models/ldamodel.html).

Además, evaluaremos estas tareas usando métricas propias de la tarea.
* [Coherence](https://radimrehurek.com/gensim/models/coherencemodel.html): [source](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf)

Corpus:

In [ ]:
!gdown 19Kp2DpVtPVe4kDN7yhNwlquiteUfqaFP

Downloading...
From: https://drive.google.com/uc?id=19Kp2DpVtPVe4kDN7yhNwlquiteUfqaFP
To: /content/topic_modelling_dataset.csv
100% 9.11M/9.11M [00:00<00:00, 106MB/s]


In [ ]:
import pandas as pd

tm_df = pd.read_csv('topic_modelling_dataset.csv')
tm_df

# Instrucciones

1. Hacer un preprocesamiento básico que consideres necesario.
2. Realizar las vectorizaciones BOW y TF-IDF
3. Realiza clustering con K-Means, escogiendo el valor de $K$ dado por el valor de codo.
4. Evalua ambas tareas de clustering con el score de Silueta.
5. Con el mismo número de tópicos del punto 3, haz topic modelling con LSA (escoge BOW o TF-IDF).
6. Evalua la tarea con la métrica Coherencia UMASS (usa la implementación de abajo).

In [ ]:
#@title Implementación coherencia `u_mass`

import math

def get_umass_score(dt_matrix, i, j):
    zo_matrix = (dt_matrix > 0).astype(int)
    col_i, col_j = zo_matrix[:, i], zo_matrix[:, j]
    col_ij = col_i + col_j
    col_ij = (col_ij == 2).astype(int)
    Di, Dij = col_i.sum(), col_ij.sum()
    return math.log((Dij + 1) / Di)

def get_topic_coherence(dt_matrix, topic, n_top_words):
    indexed_topic = zip(topic, range(0, len(topic)))
    topic_top = sorted(indexed_topic, key=lambda x: 1 - x[0])[0:n_top_words]
    coherence = 0
    for j_index in range(0, len(topic_top)):
        for i_index in range(0, j_index - 1):
            i = topic_top[i_index][1]
            j = topic_top[j_index][1]
            coherence += get_umass_score(dt_matrix, i, j)
    return coherence

def get_average_topic_coherence(dt_matrix, topics, n_top_words):
    total_coherence = 0
    for i in range(0, len(topics)):
        total_coherence += get_topic_coherence(dt_matrix, topics[i], n_top_words)
    return total_coherence / len(topics)

Ejemplo de uso:

        get_average_topic_coherence(dt_matrix, topics, n_top_words)

* `dt_matrix`: Matriz de vectorizaciones TF-IDF o BOW.
* `topics`: Matriz almacenada en el atributo `components_` de `TruncatedSVD`.
* `n_top_words`: Cuántas palabras más frecuentes tomar en cada tópico para evaluar la coherencia.

Recordar que es un número negativo, entre más alto es mejor.